In [107]:
import pandas as pd
import re

In [108]:
year = "1963"

In [109]:
df_stops = pd.read_csv(f"{year}-stops-localised-final.csv")  

In [110]:
df_stops['stop_name'] = df_stops['stop_name'].astype(str).str.strip()
df_stops['location'] = df_stops['location'].astype(str).str.strip()

In [111]:
df_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407 entries, 0 to 406
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   stop_name   407 non-null    object
 1   type        407 non-null    object
 2   in_lines    407 non-null    object
 3   location    407 non-null    object
 4   identifier  117 non-null    object
dtypes: object(5)
memory usage: 16.0+ KB


In [112]:
df_stops = df_stops.fillna('').groupby(['stop_name', 'type', 'location']).agg({
    'in_lines': ', '.join,
    'identifier': ', '.join
}).reset_index()

# Reset the index
df_stops.reset_index(inplace=True)
df_stops.rename(columns={'index': 'stop_id'}, inplace=True)

df_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   stop_id     396 non-null    int64 
 1   stop_name   396 non-null    object
 2   type        396 non-null    object
 3   location    396 non-null    object
 4   in_lines    396 non-null    object
 5   identifier  396 non-null    object
dtypes: int64(1), object(5)
memory usage: 18.7+ KB


In [113]:
df_line = pd.read_csv(f"../initial/line_data_{year}_east.csv")  

In [114]:
# Replace U+00a0 with regular whitespace in the entire DataFrame
df_line.replace('\u00a0', ' ', regex=True, inplace=True)

In [115]:
df_line.head(2)

,Unnamed: 0,line_name,type,east-west,Stops,Frequency_7-30,Length (time),direction,Length (km),year,Info
0,0,A,u-bahn,east,Pankow (Vinetastrasse) - Schönhauser Allee - D...,2,19,both,NaN,1963,NaN
1,1,E,u-bahn,east,Alexanderplatz - Schillingstrasse - Strausberg...,5,15,both,NaN,1963,NaN


In [116]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
df_stops['stop_id'] = year + df_stops['stop_id'].astype(str)
df_stops.head(2)

,stop_id,stop_name,type,location,in_lines,identifier
0,19630,"Ahrensfelde, Kirche",Autobus,"52.57532251281871, 13.576456279814597",['A 46'],
1,19631,Alexanderplatz,u-bahn,"52.5214,13.4119","['E', 'A']",Q17173458


In [117]:
df_line['line_name'] = df_line['line_name'].astype(str).str.strip()
df_line['Stops'] = df_line['Stops'].astype(str).str.strip()
df_line['year'] = df_line['year'].astype(int)
df_line['Frequency_7-30	'] = pd.to_numeric(df_line['Frequency_7-30'], errors='coerce').fillna(0).astype(int)
df_line['Length (time)'] = pd.to_numeric(df_line['Length (time)'], errors='coerce').fillna(0).astype(int)
df_line['direction'] = df_line['direction'].astype(str).str.strip()

In [118]:
# Define a function to remove leftover parentheses from a column
def remove_double_whitespace(text):
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    text = text.replace("–", "-")
    text = text.replace(" - ", " - ")
    return text

# Apply the function to the 'line_stops' column
df_line['Stops'] = df_line['Stops'].apply(remove_double_whitespace)
df_line['Stops'] = df_line['Stops'].str.replace(r'\s*–\s*', ' - ', regex=True)

In [119]:
mask = df_line['Stops'].str.contains(" - ").fillna(True)
none_df = df_line[~mask]
none_df

,Unnamed: 0,line_name,type,east-west,Stops,Frequency_7-30,Length (time),direction,Length (km),year,Info,Frequency_7-30\t


In [120]:
# Filter rows where 'line_stops' is not a string
non_string_rows = df_line[~df_line['Stops'].apply(lambda x: isinstance(x, str))]
non_string_rows

,Unnamed: 0,line_name,type,east-west,Stops,Frequency_7-30,Length (time),direction,Length (km),year,Info,Frequency_7-30\t


In [121]:
### Creating related tables

In [122]:
def extract_first_start_stop(string):
    stations = string.split(" - ")
    first_station = stations[0]
    last_station = stations[-1]
    return f"{first_station}<> {last_station}"

In [123]:
def create_line_df(df):
    line_df = pd.DataFrame({
        'line_id': range(1, 1+len(df)),
        'year': df['year'],
        'line_name': df['line_name'],
        'type': df["type"],
        "start_stop": df['Stops'].apply(extract_first_start_stop),
        "Length (time)": df['Length (time)'],
        "east_west": df['east-west'],
        "Frequency_7-30": df['Frequency_7-30'],
        "direction": df["direction"]
    })
    return line_df

In [124]:
line_df = create_line_df(df_line)

In [125]:
line_df.head(2)

,line_id,year,line_name,type,start_stop,Length (time),east_west,Frequency_7-30,direction
0,1,1963,A,u-bahn,Pankow (Vinetastrasse)<> Thälmannplatz,19,east,2,both
1,2,1963,E,u-bahn,Alexanderplatz<> Friedrichsfelde (Tierpark),15,east,5,both


In [126]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
line_df['line_id'] = year + line_df['line_id'].astype(str)
line_df.head(2)

,line_id,year,line_name,type,start_stop,Length (time),east_west,Frequency_7-30,direction
0,19631,1963,A,u-bahn,Pankow (Vinetastrasse)<> Thälmannplatz,19,east,2,both
1,19632,1963,E,u-bahn,Alexanderplatz<> Friedrichsfelde (Tierpark),15,east,5,both


In [127]:
def create_line_stops_df(df):
    line_stops = df['Stops'].str.split(' - ', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='stop_name')

    line_stops['stop_order'] = line_stops.groupby('index').cumcount()
    #index starts from 0 so it looks like 1 row is missing but this is not true

    # Clean the 'Stop Name' column by removing whitespace and non-breaking spaces
    line_stops['stop_name'] = line_stops['stop_name'].str.replace(u'\xa0', ' ').str.strip()

    # reset index so that it can be used for foreign key

    return line_stops

In [128]:
line_stops_df = create_line_stops_df(df_line)

In [129]:
for index, row in line_stops_df.iterrows():
    line_stops_df.at[index, "index"] = row["index"] + 1
line_stops_df.rename(columns={'index': 'line_id'}, inplace=True)

line_stops_df

,line_id,stop_name,stop_order
0,1,Pankow (Vinetastrasse),0
1,1,Schönhauser Allee,1
2,1,Dimitroffstrasse,2
3,1,Luxemburgplatz,3
4,1,Alexanderplatz,4
...,...,...,...
608,68,Bhf. Eichwalde,2
609,69,Alt-Schmöckwitz,0
610,69,Konsum Wernsdorf,1
611,69,Konsum Ziegenhals,2


In [130]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
line_stops_df['line_id'] = year + line_stops_df['line_id'].astype(str)
line_stops_df.head(2)

,line_id,stop_name,stop_order
0,19631,Pankow (Vinetastrasse),0
1,19631,Schönhauser Allee,1


In [131]:
# get a list of unique Line IDs in the line_stops DataFrame
line_ids = line_stops_df['line_id'].unique()

# loop through each unique Line ID
for line_id in line_ids:
    # get the first and last Stop Name for this Line ID
    first_stop_name = line_stops_df.loc[line_stops_df['line_id'] == line_id, 'stop_name'].iloc[0]
    last_stop_name = line_stops_df.loc[line_stops_df['line_id'] == line_id, 'stop_name'].iloc[-1]
    
    # get the corresponding Start-Stop value from line_df for this Line ID
    start_stop = line_df.loc[line_df['line_id'] == line_id, 'start_stop'].iloc[0]
    
    # check if the first and last Stop Names are contained in the Start-Stop value
    if first_stop_name not in start_stop or last_stop_name not in start_stop:
        print(f"Quality control check failed for line_id {line_id}.")

# I checked that this works using the following code:
# line_stops_df.loc[0, "stop_name"] = "test"
# and got the right result: Quality control check failed for line_id 1001.


In [132]:
def add_type(line_stops, line_df):
    # Assuming line_id is the common column between line_stops and line_df
    merged_df = pd.merge(line_stops, line_df[['line_id', 'type', "line_name"]], on='line_id', how='left')
    
    # Rename the 'type' column from line_df to 'type_from_line_df' to avoid conflicts
    merged_df.rename(columns={'type': 'type'}, inplace=True)
    
    # Drop the 'type_from_line_df' column if it's not needed in the final result
    # merged_df.drop(columns=['type_from_line_df'], inplace=True)
    
    return merged_df

In [133]:
line_stops_df = add_type(line_stops_df, line_df)

In [134]:
line_stops_df.loc[120:130]

,line_id,stop_name,stop_order,type,line_name
120,196313,S-Bhf. Pankow,5,strassenbahn,46
121,196313,U-Bhf. Pankow (Vinetastrasse),6,strassenbahn,46
122,196313,Bornholmer Strasse Ecke Schönhauser Allee,7,strassenbahn,46
123,196313,S-Bhf. Schönhauser Allee,8,strassenbahn,46
124,196313,U-Bhf. Dimitroffstrasse,9,strassenbahn,46
125,196313,Brunnenstrasse Ecke Invalidenstrasse,10,strassenbahn,46
126,196313,Oranienburger Tor,11,strassenbahn,46
127,196313,Bhf. Friedrichstrasse,12,strassenbahn,46
128,196313,Am Kupfergraben,13,strassenbahn,46
129,196314,"Bucholz, Hauptstrasse",0,strassenbahn,49


In [135]:
df_stops

,stop_id,stop_name,type,location,in_lines,identifier
0,19630,"Ahrensfelde, Kirche",Autobus,"52.57532251281871, 13.576456279814597",['A 46'],
1,19631,Alexanderplatz,u-bahn,"52.5214,13.4119","['E', 'A']",Q17173458
2,19632,"Alexanderplatz, Grunerstrasse",Omnibus,"52.52008326268696, 13.41408418961877",['1'],
3,19633,"Alexanderplatz, Grunerstrasse",strassenbahn,"52.52008326268696, 13.41408418961877",['71'],
4,19634,"Alexanderplatz, Leninallee",strassenbahn,"52.522103158947594, 13.416511130432587","['69', '64']",
...,...,...,...,...,...,...
391,1963391,Wolfmarsteig,Autobus,"52.41268952057738, 13.536602630335091",['A 23'],
392,1963392,Woltersdorfer Schleuse,Autobus,"52.4427723984596, 13.764524016423056",['S-Bhf. Rahnsdorf - Woltersdorfer Schleuse'],
393,1963393,Wongrowitzer Steig,Autobus,"52.4669377,13.5853227",['A 27'],
394,1963394,Zentralschule,Autobus,"52.43465069286089, 13.703692212644038",['A 22'],


In [136]:
def add_fk(line_stops_df, df_stops):
    # Create a new dataframe with the Stop Name and Stop ID columns
    stop_id_df = df_stops[['stop_name', 'stop_id', 'type', 'in_lines']]

    # Merge the line_stops_df and stop_id_df dataframes based on matching stop names and line_name condition
    line_stops_df = line_stops_df.merge(stop_id_df,
                                        left_on=['stop_name', 'type'],
                                        right_on=['stop_name', 'type'],
                                        how='left')

    # Filter the rows based on the condition that line_name is contained in in_lines
    line_stops_df = line_stops_df[line_stops_df.apply(lambda row: str(row['line_name']) in str(row['in_lines']), axis=1)]

    return line_stops_df

# Assuming line_stops_df and df_stops are your dataframes
# Replace 'stop_name', 'stop_id', 'type', and 'in_lines' with the actual column names you have

line_stops_df = add_fk(line_stops_df, df_stops)


In [137]:
# Calculate the difference between consecutive 'stop_order' values
line_stops_df['diff'] = line_stops_df['stop_order'].diff()

# Use the following line to drop rows where the 'diff' column is 0.0
line_stops_df = line_stops_df[line_stops_df['diff'] != 0.0]

# Identify faulty rows where the difference is not 1 digit behind
faulty_rows = line_stops_df[(line_stops_df['diff'] != 1) & (line_stops_df['stop_order'] != 0)]
faulty_rows

,line_id,stop_name,stop_order,type,line_name,stop_id,in_lines,diff


In [138]:
faulty_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   line_id     0 non-null      object 
 1   stop_name   0 non-null      object 
 2   stop_order  0 non-null      int64  
 3   type        0 non-null      object 
 4   line_name   0 non-null      object 
 5   stop_id     0 non-null      object 
 6   in_lines    0 non-null      object 
 7   diff        0 non-null      float64
dtypes: float64(1), int64(1), object(6)
memory usage: 0.0+ bytes


In [139]:
# convert 'Stop ID' column to numeric values, coercing errors to NaN
line_stops_df['stop_id'] = pd.to_numeric(line_stops_df['stop_id'], errors='coerce')

# check if all values in 'Stop ID' column are numeric
if line_stops_df['stop_id'].notnull().all():
    print("All values in 'stop_id' column are numeric")
else:
    print("There are non-numeric values in 'stop_id' column")
    print(line_stops_df[line_stops_df['stop_id'].isnull()])


All values in 'stop_id' column are numeric


In [140]:
line_stops_df.drop(['line_name', "in_lines", "type", "diff"], axis=1, inplace=True)

In [142]:
line_stops_df.to_csv(f"line_stops_{year}-final.csv")
line_df.to_csv(f"line_df_{year}-final.csv")
df_stops.to_csv(f"stops_df_{year}-final.csv")